## Generate finder charts for SALT RSS
Includes automatic alignment star selection and position angle computation for faint targets.

In [ ]:
%load_ext lab_black

In [ ]:
import numpy as np
import astropy as ap
import regions
import astropy.coordinates as apcoords
import astropy.units as apunits
import astropy.io.fits as apfits
import astropy.table as aptable
import astropy.visualization as apviz
import astropy.wcs as apwcs
import astropy.time as aptime
import matplotlib.pyplot as mplplot
import pandas as pd

Expects a CSV file with at least the following columns:
* target name
* target type
* optional
* right ascension
* declination
* equinox
* bandpass
* minimum magnitude
* maximum magnitude

In [ ]:
targetDataCsv = ??

In [ ]:
targetData = pd.read_csv(targetDataCsv)

In [ ]:
targetCoords = apcoords.SkyCoord(
    ra=targetData["right ascension"], dec=targetData.declination, unit="deg"
)

In [ ]:
targetData = pd.concat(
    [
        targetData,
        pd.DataFrame(
            np.array(
                [
                    targetCoords.ra.to_string(sep=" ", pad=False, unit="hour"),
                    targetCoords.dec.to_string(sep=" ", alwayssign=True, pad=False),
                ]
            ).T,
            columns=["ra", "dec"],
        ),
    ],
    axis=1,
)

Specify a subset of the targets in your CSV file

In [ ]:
targets = [??]

In [ ]:
targetData.set_index("target name").loc[targets]

Specify a directory to store data

In [ ]:
dataDir = ??

This saves a file in the format required to download a set of FITS images from `http://www-wfau.roe.ac.uk/sss/batchfile.html`

In [ ]:
with open(f"{dataDir}/requiredChartObjCoords.txt", mode="w") as coordFile:
    coordFile.write(
        "\n".join(
            [
                " ".join(t)
                for t in targetData.set_index("target name")
                .loc[targets, ["ra", "dec"]]
                .to_numpy()
            ]
        )
    )

Use file to download data from `http://www-wfau.roe.ac.uk/sss/batchfile.html` before proceeding.

This should copy the files you donloaded and name them according to their corresponding targets. May be a bit unreliable.

In [ ]:
import shutil
import glob
import os

if not os.path.exists(f"{dataDir}/fits"):
    os.mkdir(f"{dataDir}/fits")

fitsFiles = sorted(glob.glob(f"{dataDir}/fits/*.fits"))
for target, file in zip(targets, fitsFiles):
    shutil.copy(file, os.path.join(os.path.dirname(file), f"{target}.fits"))

### The actual plotting function

* If `plotSelection==True` then all considered alignment stars will be plotted. 
* Specify an angle in degrees for `forcedPA` to disable automatic PA specification. 
* Use `customObjects` to specify a `dict` of label-`astropy.SkyCoord` key-value pairs to plot custom object locations.

In [ ]:
def plotTarget(target, plotSelection=True, forcedPA=None, customObjects=None):

    targetCoords = targetData.set_index("target name").loc[
        target, ["right ascension", "declination"]
    ]

    with apfits.open(f"{dataDir}/fits/{target}.fits") as targetImageFits:
        newHeader = targetImageFits["PRIMARY"].header.copy()

        for kw in targetImageFits["PRIMARY"].header:
            if "PIXELSZ" in kw:
                newHeader.remove(kw)

        ax = mplplot.figure(figsize=(15, 15)).add_subplot(
            1, 1, 1, projection=apwcs.WCS(newHeader, fix=True)
        )
        ax.imshow(
            targetImageFits["PRIMARY"].data,
            norm=apviz.ImageNormalize(
                stretch=apviz.LinearStretch(),  # HistEqStretch(data=targetImageFits["PRIMARY"].data),
                interval=apviz.ZScaleInterval(),
            ),
            cmap="Greys",
        )
        catalogue = aptable.Table(targetImageFits[1].data).to_pandas()
        selection = catalogue.loc[(catalogue.R_2 < 17) & (catalogue.R_2 > 12)]
        if forcedPA is None:
            if plotSelection:
                selection.plot.scatter(
                    x="RA",
                    y="DEC",
                    ax=ax,
                    transform=ax.get_transform("world"),
                    s=300,
                    fc="none",
                    c=selection.R_2,
                    cmap="viridis",
                    colorbar=False,
                )
        targetFrame = targetData.set_index("target name").loc[target, :].to_frame().T
        targetFrame.plot.scatter(
            x="right ascension",
            y="declination",
            fc="none",
            s=300,
            label=target,
            ec="r",
            ax=ax,
            transform=ax.get_transform("world"),
        )

        selectionCoords = apcoords.SkyCoord(
            ra=selection.RA, dec=selection.DEC, unit="deg"
        )
        targetCoord = apcoords.SkyCoord(
            ra=targetFrame["right ascension"], dec=targetFrame.declination, unit="deg"
        )
        if forcedPA is None:
            closestIndex = selectionCoords.separation(targetCoord).argmin()
            closestCoord = selectionCoords[closestIndex]
            closestFrame = selection.iloc[closestIndex].to_frame().T
            closestFrame["obs_time"] = obsTime = aptime.Time(
                newHeader["MJD-OBS"], format="mjd"
            ).isot
            closestFrame.plot.scatter(
                x="RA",
                y="DEC",
                marker="s",
                s=500,
                label="Alignment Star",
                fc="none",
                ec="r",
                ax=ax,
                transform=ax.get_transform("world"),
            )

            pa = targetCoord.position_angle(closestCoord).to("deg").value[0]
        else:
            pa = forcedPA
            closestFrame = pd.DataFrame(columns=selection.columns)

        if customObjects is not None:
            for counter, (label, coord) in enumerate(customObjects.items()):
                ax.scatter(
                    x=coord.ra,
                    y=coord.dec,
                    marker="s",
                    s=500,
                    label=label,
                    fc="none",
                    ec="r",
                    transform=ax.get_transform("world"),
                )

        regions.RectangleSkyRegion(
            center=targetCoord[0],
            width=3 * apunits.arcsec,
            height=8 * apunits.arcmin,
            angle=pa * apunits.deg,
        ).to_pixel(wcs=apwcs.WCS(newHeader, fix=True)).plot(ax=ax)

        regions.CircleSkyRegion(
            center=targetCoord[0], radius=4 * apunits.arcmin
        ).to_pixel(wcs=apwcs.WCS(newHeader, fix=True)).plot(ax=ax, ec="b")

        regions.CircleSkyRegion(
            center=targetCoord[0], radius=5 * apunits.arcmin
        ).to_pixel(wcs=apwcs.WCS(newHeader, fix=True)).plot(ax=ax, ec="b")

        ax.legend(ncol=2, loc="lower center", fontsize="x-large")

        ax.text(
            0.95,
            -0.05,
            f"PA = {pa:.1f}",
            transform=ax.transAxes,
            style="italic",
            weight="bold",
            fontsize="xx-large",
        )

        ax.text(
            -0.05,
            -0.05,
            "POSS2/UKSTU Blue",
            transform=ax.transAxes,
            style="italic",
            weight="bold",
            fontsize="xx-large",
        )
        ax.text(
            0.79,
            0.79,
            "RSS",
            transform=ax.transAxes,
            style="italic",
            weight="bold",
            size="large",
            horizontalalignment="left",
            color="b",
            fontsize="xx-large",
        )
        ax.text(
            0.86,
            0.86,
            "SCAM",
            transform=ax.transAxes,
            style="italic",
            weight="bold",
            size="large",
            horizontalalignment="left",
            color="b",
            fontsize="xx-large",
        )

        ax.text(
            targetCoord.ra.value[0],
            targetCoord.dec.value[0] + 4.8 / 60.0,
            "N",
            style="italic",
            weight="bold",
            size="large",
            color=(0, 0.5, 1),
            transform=ax.get_transform("world"),
            fontsize="xx-large",
        )
        ax.text(
            targetCoord.ra.value[0]
            + 4.8 / (np.abs(np.cos(targetCoord.dec.value[0] * np.pi / 180.0)) * 60),
            targetCoord.dec.value[0],
            "E",
            style="italic",
            weight="bold",
            size="large",
            horizontalalignment="right",
            color=(0, 0.5, 1),
            transform=ax.get_transform("world"),
            fontsize="xx-large",
        )

        ax.set_title(f"{target}", style="italic", weight="bold", fontsize="xx-large")
        ax.set_ylabel("Dec. (ICRS)", fontsize="xx-large")
        ax.set_xlabel("RA. (ICRS)", fontsize="xx-large")
        lon, lat = (ax.coords[0], ax.coords[1])
        lon.display_minor_ticks(True)
        lat.display_minor_ticks(True)
        lon.set_ticklabel(size="x-large")
        lat.set_ticklabel(size="x-large")

    mplplot.savefig(f"{dataDir}/{target}.pdf", bbox_inches="tight", rasterize=True)
    return closestFrame

Example useage.

In [ ]:
plotSelection = False

customObjects = {
    "Actual Target": apcoords.SkyCoord(ra="13h57m01.95s", dec="-47d48m10.0s")
}

closestFrame_ = pd.concat(
    [
        plotTarget(target, plotSelection, forcedPA=0, customObjects=customObjects)
        for target in targets
    ]
)